In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os 

In [3]:
os.chdir('/content/drive/My Drive/Assignment')

In [4]:
!ls

processed_test.csv   RFC_Model_all.pckl  test_indessa.csv   Untitled.ipynb
processed_train.csv  RFC_Model.pckl	 train_indessa.csv  XGB.pckl


In [5]:
import pandas as pd
import numpy as np
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt

In [6]:
#import processed data
data=pd.read_csv('processed_train.csv')

In [7]:
#check for data cleanliness
data.head()

,funded_amnt_inv,int_rate,grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_rec_int,total_rec_late_fee,recoveries,collections_12_mths_ex_med,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,loan_status,term_int,sub_grade_int,last_week_pay_int
0,14350.0,19.19,4,319.0,9.0,4,28700.0,1,0,1,349xx,9,33.88,0.0,1.0,1.0,22515.0,73.1,28.0,0,1173.84,0.0,0.0,0.0,0,0.0,0.0,28699.0,0.0,36.0,3.0,26.0
1,4800.0,10.99,1,76.0,0.0,1,65000.0,1,0,2,209xx,20,3.64,0.0,1.0,0.0,7624.0,23.2,13.0,1,83.95,0.0,0.0,0.0,0,0.0,0.0,9974.0,0.0,36.0,4.0,9.0
2,10000.0,7.26,0,2309.0,2.0,4,45000.0,0,0,1,447xx,35,18.42,0.0,0.0,0.0,10877.0,31.2,19.0,1,56.47,0.0,0.0,0.0,0,0.0,65.0,38295.0,0.0,36.0,4.0,9.0
3,15000.0,19.72,3,1.0,10.0,5,105000.0,0,0,1,221xx,45,14.97,0.0,2.0,0.0,13712.0,55.5,21.0,0,4858.62,0.0,0.0,0.0,0,0.0,0.0,55564.0,0.0,36.0,5.0,135.0
4,16000.0,10.64,1,1.0,10.0,5,52000.0,2,0,0,900xx,4,20.16,0.0,0.0,0.0,35835.0,76.2,27.0,1,2296.41,0.0,0.0,0.0,0,0.0,0.0,47159.0,0.0,36.0,2.0,96.0


In [8]:
data.isna().sum()

funded_amnt_inv               0
int_rate                      0
grade                         0
emp_title                     0
emp_length                    0
home_ownership                0
annual_inc                    0
verification_status           0
pymnt_plan                    0
purpose                       0
zip_code                      0
addr_state                    0
dti                           0
delinq_2yrs                   0
inq_last_6mths                0
pub_rec                       0
revol_bal                     0
revol_util                    0
total_acc                     0
initial_list_status           0
total_rec_int                 0
total_rec_late_fee            0
recoveries                    0
collections_12_mths_ex_med    0
application_type              0
acc_now_delinq                0
tot_coll_amt                  0
tot_cur_bal                   0
loan_status                   0
term_int                      0
sub_grade_int                 0
last_wee

In [9]:
#lets create dependent and independant parameters 

Y= data['loan_status']
X=data.drop(['loan_status', 'zip_code'], axis=1)
print(X.shape, Y.shape)

(532428, 30) (532428,)


In [10]:
### we need to do feature selection before running the models 

from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

In [11]:
#scale the features before putting it into lasso feature selction 
scaler = StandardScaler()
scaler.fit(X,Y)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [12]:
X1= scaler.transform(X)
X.shape

(532428, 30)

In [13]:
X1= pd.DataFrame(X1)
X1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,-0.042047,1.357892,1.678774,-0.154784,0.815511,0.571140,-0.710585,-0.034906,-0.003876,-0.210536,-0.932853,1.880883,-0.365624,0.306313,1.379109,0.249461,0.756608,0.230739,-0.971103,-0.276892,-0.096529,-0.111603,-0.107519,-0.023941,-0.063387,-0.113614,-0.750447,-0.65438,0.021901,-0.693461
1,-1.173391,-0.514423,-0.607492,-0.313174,-1.640007,-0.990027,-0.153833,-0.034906,-0.003876,0.317881,-0.206855,-1.732423,-0.365624,0.306313,-0.333769,-0.414628,-1.335900,-1.035830,1.029757,-0.797573,-0.096529,-0.111603,-0.107519,-0.023941,-0.063387,-0.113614,-0.877208,-0.65438,0.729327,-1.074467
2,-0.557371,-1.366097,-1.369581,1.142313,-1.094337,0.571140,-0.460583,-1.296970,-0.003876,-0.210536,0.783143,0.033604,-0.365624,-0.696686,-0.333769,-0.269555,-1.000428,-0.529203,1.029757,-0.810702,-0.096529,-0.111603,-0.107519,-0.023941,-0.063387,-0.079034,-0.685486,-0.65438,0.729327,-1.074467
3,0.034956,1.478907,0.916686,-0.362059,1.088346,1.091529,0.459668,-1.296970,-0.003876,-0.210536,1.443142,-0.378628,-0.365624,1.309312,-0.333769,-0.143123,0.018569,-0.360327,-0.971103,1.483468,-0.096529,-0.111603,-0.107519,-0.023941,-0.063387,-0.113614,-0.568581,-0.65438,1.436753,1.749461
4,0.153421,-0.594338,-0.607492,-0.362059,1.088346,1.091529,-0.353221,1.227157,-0.003876,-0.738952,-1.262852,0.241512,-0.365624,-0.696686,-0.333769,0.843489,0.886603,0.146301,1.029757,0.259403,-0.096529,-0.111603,-0.107519,-0.023941,-0.063387,-0.113614,-0.625480,-0.65438,-0.685526,0.875388


In [ ]:
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1',solver='saga',max_iter=500))
sel_.fit(X1, Y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=500, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='saga',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [ ]:
sel_.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [ ]:
selected_feat = X.columns[(sel_.get_support())]
print('total features: {}'.format((X.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
      np.sum(sel_.estimator_.coef_ == 0)))

total features: 30
selected features: 30
features with coefficients shrank to zero: 0


In [11]:
## to be more certain, lets do it feature selection with Random Forste as well

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [ ]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))

In [ ]:
sel.fit(X1,Y)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                                 class_weight=None,
                                                 criterion='gini',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100, n_jobs=None,
                                                 oob_score=False,

In [ ]:
selected_feat= X.columns[(sel.get_support())]

In [ ]:
print(selected_feat)

Index(['funded_amnt_inv', 'int_rate', 'emp_title', 'annual_inc', 'addr_state',
       'dti', 'revol_bal', 'revol_util', 'total_acc', 'total_rec_int',
       'tot_coll_amt', 'tot_cur_bal', 'last_week_pay_int'],
      dtype='object')


In [ ]:
len(selected_feat)

13

In [ ]:
##try fiting the data without scalling
sel.fit(X,Y)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                                 class_weight=None,
                                                 criterion='gini',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100, n_jobs=None,
                                                 oob_score=False,

In [ ]:
selected_feat_noscal= X.columns[(sel.get_support())]

In [ ]:
print(selected_feat, len(selected_feat_noscal))

Index(['funded_amnt_inv', 'int_rate', 'emp_title', 'annual_inc', 'addr_state',
       'dti', 'revol_bal', 'revol_util', 'total_acc', 'total_rec_int',
       'tot_coll_amt', 'tot_cur_bal', 'last_week_pay_int'],
      dtype='object') 13


In [15]:
selected_feat_noscal= ['funded_amnt_inv', 'int_rate', 'emp_title', 'annual_inc', 'addr_state',
       'dti', 'revol_bal', 'revol_util', 'total_acc', 'total_rec_int',
       'tot_coll_amt', 'tot_cur_bal', 'last_week_pay_int']

In [ ]:
#lets run two different models with 30 features and 13 features 

In [18]:
#lets try with random Forest 

In [12]:
#try random forest with selected parametres
rfc_new = RandomForestClassifier(n_jobs=-1, criterion='gini',max_depth=10, max_features='sqrt',n_estimators=700) 

In [13]:
## lets do cross validation model
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

In [19]:
Xtrain, Xtest, Ytrain,Ytest = train_test_split(X,Y,test_size=0.25, random_state=42 )
print(Xtrain.shape,Xtest.shape, Ytrain.shape,Ytest.shape)

(399321, 30) (133107, 30) (399321,) (133107,)


In [25]:
#cv = cross_validate(rfc_new, X_13, Y, cv=10)

In [27]:
rfc_new.fit(Xtrain,Ytrain)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=700,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [34]:
Ypred= rfc_new.predict(Xtest)

In [36]:
from sklearn import metrics

In [36]:
Ypred


array([0., 0., 0., ..., 0., 0., 0.])

In [37]:
print(metrics.confusion_matrix(Ytest,Ypred))
print(metrics.classification_report(Ytest, Ypred))
print(metrics.accuracy_score(Ytest,Ypred))

[[100987    690]
 [ 22004   9426]]
              precision    recall  f1-score   support

         0.0       0.82      0.99      0.90    101677
         1.0       0.93      0.30      0.45     31430

    accuracy                           0.83    133107
   macro avg       0.88      0.65      0.68    133107
weighted avg       0.85      0.83      0.79    133107

0.8295055857317797


In [102]:
Ypred1= rfc_new.predict_proba(Xtest)

In [104]:
Ypred2=Ypred1

In [105]:
for i in range(len(Ypred1)):
  if Ypred2[i][1]>=0.3:
    Ypred2[i][1]=int(1)
  else:
    Ypred2[i][1]=int(0)

In [106]:
Ypred11=Ypred2[:,1]


In [107]:
print(metrics.confusion_matrix(Ytest,Ypred11))
print(metrics.classification_report(Ytest, Ypred11))
print(metrics.accuracy_score(Ytest,Ypred11))

[[94483  7194]
 [16560 14870]]
              precision    recall  f1-score   support

         0.0       0.85      0.93      0.89    101677
         1.0       0.67      0.47      0.56     31430

    accuracy                           0.82    133107
   macro avg       0.76      0.70      0.72    133107
weighted avg       0.81      0.82      0.81    133107

0.8215420676598526


In [17]:
## as its not giving proper accuracy , lets do SMOT oversampling 
from imblearn.over_sampling import SMOTE
from collections import Counter

In [18]:
counter= Counter(Ytrain)
print("before", counter)

smt=SMOTE()

Xtrain_sm, Ytrain_sm= smt.fit_resample(Xtrain,Ytrain)

counter= Counter(Ytrain_sm)

print('after', counter)

before Counter({0.0: 304924, 1.0: 94397})


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


after Counter({0.0: 304924, 1.0: 304924})


In [20]:
print(Xtrain.shape,Xtrain_sm.shape, Ytrain.shape,Ytrain_sm.shape)

(399321, 30) (609848, 30) (399321,) (609848,)


In [ ]:
#lets train XG boost on oversampled data

In [59]:
import pickle

In [ ]:
filename = 'RFC_Model.pckl' 
pickle.dump(rfc_new, open(filename, 'wb'))

In [21]:
#lets now try with XGboost classifier 
from xgboost import XGBClassifier

In [22]:
XGB= XGBClassifier()

In [23]:
XGB.fit(Xtrain_sm,Ytrain_sm)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [31]:
new_col=['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29']
old_col=['funded_amnt_inv', 'int_rate', 'grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'pymnt_plan', 'purpose', 'addr_state', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collections_12_mths_ex_med', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'term_int', 'sub_grade_int', 'last_week_pay_int']
res = {}
for key in old_col:
    for value in new_col:
        res[key] = value
        new_col.remove(value)
        break  
#Xtest.rename(columns =[new_col], inplace = True)

In [32]:
Xtest.rename(columns =res, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [33]:
Xtest.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29
447404,5000.0,13.18,2,1905.0,5.0,1,36500.0,0,0,8,43,13.48,1.0,0.0,0.0,19580.0,65.0,15.0,0,47.60,0.0,0.0,0.0,0,0.0,1418.000000,19580.000000,36.0,3.0,4.0
105437,30000.0,12.69,2,1.0,7.0,1,165000.0,2,0,1,38,14.56,0.0,0.0,0.0,31104.0,71.5,20.0,1,1750.34,0.0,0.0,0.0,0,0.0,0.000000,443622.000000,36.0,2.0,26.0
418519,35000.0,15.61,3,367.0,10.0,5,103590.0,2,0,0,4,14.87,0.0,0.0,0.0,48903.0,98.6,12.0,1,3171.29,0.0,0.0,0.0,0,0.0,0.000000,56948.000000,60.0,1.0,31.0
264650,12000.0,19.79,4,1.0,10.0,1,167000.0,1,0,4,43,10.85,0.0,1.0,0.0,1133.0,17.2,22.0,0,5497.55,0.0,0.0,0.0,0,0.0,213.562222,139554.110792,60.0,2.0,152.0
32781,12100.0,13.67,1,1.0,4.0,5,60000.0,0,0,1,9,12.85,0.0,0.0,0.0,12026.0,90.1,11.0,0,2717.17,0.0,0.0,0.0,0,0.0,213.562222,139554.110792,36.0,5.0,156.0


In [34]:
Ypred_XGB= XGB.predict(Xtest)

In [38]:
print(metrics.confusion_matrix(Ytest,Ypred_XGB ))
print(metrics.classification_report(Ytest, Ypred_XGB))
print(metrics.accuracy_score(Ytest,Ypred_XGB))

[[98756  2921]
 [20009 11421]]
              precision    recall  f1-score   support

         0.0       0.83      0.97      0.90    101677
         1.0       0.80      0.36      0.50     31430

    accuracy                           0.83    133107
   macro avg       0.81      0.67      0.70    133107
weighted avg       0.82      0.83      0.80    133107

0.8277325760478412


In [39]:
##try simple logistic regression 
from sklearn.linear_model import  LogisticRegression

In [40]:
logreg= LogisticRegression(max_iter=1000)

In [41]:
logreg.fit(Xtrain_sm,Ytrain_sm)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [42]:
Ypred_logreg=logreg.predict(Xtest)

In [43]:
print(metrics.confusion_matrix(Ytest,Ypred_logreg ))
print(metrics.classification_report(Ytest, Ypred_logreg))
print(metrics.accuracy_score(Ytest,Ypred_logreg))

[[73501 28176]
 [13217 18213]]
              precision    recall  f1-score   support

         0.0       0.85      0.72      0.78    101677
         1.0       0.39      0.58      0.47     31430

    accuracy                           0.69    133107
   macro avg       0.62      0.65      0.62    133107
weighted avg       0.74      0.69      0.71    133107

0.6890246192912469


In [44]:
#Lets try scaled features for Log reg#
from sklearn.preprocessing import  StandardScaler

In [45]:
sc= StandardScaler()

In [46]:
Xtrain_sm_sc= sc.fit_transform(Xtrain_sm)

In [48]:
logreg_sc= LogisticRegression()
logreg_sc.fit(Xtrain_sm_sc,Ytrain_sm)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
Ypred_log_sc= logreg_sc.predict(Xtest_sc)

In [47]:
Xtest_sc=sc.transform(Xtest)

In [50]:
print(metrics.confusion_matrix(Ytest,Ypred_log_sc ))
print(metrics.classification_report(Ytest, Ypred_log_sc))
print(metrics.accuracy_score(Ytest,Ypred_log_sc))

[[75153 26524]
 [ 9032 22398]]
              precision    recall  f1-score   support

         0.0       0.89      0.74      0.81    101677
         1.0       0.46      0.71      0.56     31430

    accuracy                           0.73    133107
   macro avg       0.68      0.73      0.68    133107
weighted avg       0.79      0.73      0.75    133107

0.7328765579571322


In [51]:
#try for Random forest on scaled data
rfc_sc= RandomForestClassifier(n_jobs=-1, criterion='gini',max_depth=10, max_features='sqrt',n_estimators=700)

In [52]:
rfc_sc.fit(Xtrain_sm_sc,Ytrain_sm)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=700,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [53]:
Ypred_rfc_sc= rfc_sc.predict(Xtest_sc)

In [54]:
print(metrics.confusion_matrix(Ytest,Ypred_rfc_sc ))
print(metrics.classification_report(Ytest, Ypred_rfc_sc))
print(metrics.accuracy_score(Ytest,Ypred_rfc_sc))

[[95794  5883]
 [18927 12503]]
              precision    recall  f1-score   support

         0.0       0.84      0.94      0.89    101677
         1.0       0.68      0.40      0.50     31430

    accuracy                           0.81    133107
   macro avg       0.76      0.67      0.69    133107
weighted avg       0.80      0.81      0.79    133107

0.8136086005995177


In [55]:
##final model XGboost with scaled data
XGB_sc= XGBClassifier()

In [56]:
XGB_sc.fit(Xtrain_sm_sc,Ytrain_sm)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [57]:
Ypred_XGB_sc= XGB_sc.predict(Xtest_sc)

In [58]:
print(metrics.confusion_matrix(Ytest,Ypred_XGB_sc ))
print(metrics.classification_report(Ytest, Ypred_XGB_sc))
print(metrics.accuracy_score(Ytest,Ypred_XGB_sc))

[[98604  3073]
 [19878 11552]]
              precision    recall  f1-score   support

         0.0       0.83      0.97      0.90    101677
         1.0       0.79      0.37      0.50     31430

    accuracy                           0.83    133107
   macro avg       0.81      0.67      0.70    133107
weighted avg       0.82      0.83      0.80    133107

0.8275748082369823


In [60]:
filename = 'XGB_sc.pckl' 
pickle.dump(XGB_sc, open(filename, 'wb'))

In [ ]:
#XGB doin good on training set 
#lets try on test set

In [61]:
!ls

processed_test.csv   RFC_Model.pckl	Untitled.ipynb
processed_train.csv  test_indessa.csv	XGB.pckl
RFC_Model_all.pckl   train_indessa.csv	XGB_sc.pckl


In [65]:
test_data= pd.read_csv('processed_test.csv')


(354951, 31)

In [67]:
test_data.drop('zip_code', axis=1, inplace=True)
test_data.head()

,funded_amnt_inv,int_rate,grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_rec_int,total_rec_late_fee,recoveries,collections_12_mths_ex_med,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,term_int,sub_grade_int,last_week_pay_int
0,14000.0,16.24,3.0,168.0,4,6.0,73500.0,2.0,1.0,0.206162,36.0,28.65,1.0,1.0,0.0,29828.0,65.3,53.0,1.0,3915.61,0.0,0.0,0.0,1.0,0.0,0.0,85230.0,60,5,104
1,16000.0,9.49,2.0,22.0,5,2.0,105000.0,2.0,1.0,0.318456,28.0,31.19,0.0,0.0,0.0,4161.0,19.4,63.0,2.0,1495.06,0.0,0.0,0.0,1.0,0.0,0.0,444991.0,60,2,57
2,11050.0,15.61,4.0,31.0,10,6.0,62300.0,1.0,1.0,0.231840,17.0,21.73,1.0,0.0,1.0,12064.0,74.0,20.0,2.0,2096.21,0.0,0.0,0.0,1.0,0.0,0.0,105737.0,60,1,70
3,34700.0,12.69,3.0,1.0,10,5.0,150000.0,2.0,1.0,0.231840,5.0,17.27,0.0,0.0,0.0,32549.0,45.0,26.0,1.0,1756.31,0.0,0.0,0.0,1.0,0.0,0.0,287022.0,60,2,22
4,6500.0,6.89,1.0,791.0,5,2.0,52000.0,1.0,1.0,0.318456,19.0,20.19,0.0,0.0,0.0,5133.0,19.2,36.0,2.0,172.21,0.0,0.0,0.0,1.0,0.0,0.0,234278.0,36,3,22


In [68]:
test_data.shape

(354951, 30)

In [70]:
#lets cover it into new columns as per scaled data
new_col=['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29']
old_col=['funded_amnt_inv', 'int_rate', 'grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'pymnt_plan', 'purpose', 'addr_state', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collections_12_mths_ex_med', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'term_int', 'sub_grade_int', 'last_week_pay_int']
res = {}
for key in old_col:
    for value in new_col:
        res[key] = value
        new_col.remove(value)
        break  
#Xtest.rename(columns =[new_col], inplace = True)

In [71]:
test_data.rename(columns =res, inplace = True)

In [72]:
#lets do predictions for test data
Ypred_final= XGB_sc.predict(test_data)

In [94]:
#try prediction with lesser threshold 
Ypred_final_th= XGB_sc.predict_proba(test_data)

In [95]:
for i in range(len(Ypred_final_th)):
  if Ypred_final_th[i][1]>=0.6:
    Ypred_final_th[i][1]=int(1)
  else:
    Ypred_final_th[i][1]=int(0)

In [96]:
Ypred_final_thre_0_3=Ypred_final_th[:,1]


In [73]:
test_data_original= pd.read_csv('test_indessa.csv')

In [74]:
submission= pd.DataFrame()

In [75]:
submission['member_id']= test_data_original['member_id']

In [76]:
submission['predictions']= Ypred_final

In [100]:
submission['predictions_0.6_threshold']= Ypred_final_thre_0_3

In [103]:
submission.head()

,member_id,predictions,predictions_0.6_threshold
0,11937648,0.0,0.0
1,38983318,0.0,0.0
2,27999917,1.0,1.0
3,61514932,1.0,0.0
4,59622821,1.0,0.0


In [102]:
submission.drop('predictions_0.3_threshold',axis=1, inplace=True)


In [104]:
submission.to_csv('final_submission.csv',encoding='utf-8',index=False)

In [105]:
submission.groupby('predictions').count()

,member_id,predictions_0.6_threshold
predictions,,
0.0,206698,206698
1.0,148253,148253


In [107]:
submission.groupby('predictions_0.6_threshold').count()

,member_id,predictions
predictions_0.6_threshold,,
0.0,270914,270914
1.0,84037,84037


In [ ]:
#can be tried on only 13 features and also with hyperparameter tuning with Grid Cv
X_13 = data[selected_feat_noscal]

In [ ]:
X_13.shape

(532428, 13)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification

In [ ]:
X_13, Y = make_classification(n_samples=1000,
                           n_features=10,
                           n_informative=3,
                           n_redundant=0,
                           n_repeated=0,
                           n_classes=2,
                           random_state=0,
                           shuffle=False)

In [ ]:
rfc = RandomForestClassifier(n_jobs=-1) 


In [ ]:
param_grid = { "n_estimators"      : [250, 500,700],
           "criterion"         : ["gini", "entropy"],
           "max_features"      : [8,10],
           "max_depth"         : [20, 30, 50],
           "min_samples_split" : [2, 4] ,
           "bootstrap": [True, False]}

In [ ]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_13, Y)
print(CV_rfc.best_params_)

KeyboardInterrupt: ignored